# One-to-Many and Many-to-Many Joins - Lab

## Introduction

In this lab, you'll practice your knowledge of one-to-many and many-to-many relationships!

## Objectives

You will be able to:

* Explain one-to-many and many-to-many joins as well as implications for the size of query results
* Query data using one-to-many and many-to-many joins

## One-to-Many and Many-to-Many Joins
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connect to the Database

Include the relevant imports, then connect to the database located at `data.sqlite`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Employees and Their Offices (a One-to-One Join)

Select all of the employees including their first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [2]:
# Your code here
cur.execute("""
    SELECT e.firstName, e.lastName, o.city, o.state
    FROM employees e
    LEFT JOIN offices o ON e.officeCode = o.officeCode
    ORDER BY e.firstName, e.lastName;
""")

results = cur.fetchall()
for first, last, city, state in results:
    city_state = f"{city}, {state}" if city and state else "No office assigned"
    print(f"{first} {last} — Office: {city_state}")


Andy Fixter — Office: No office assigned
Anthony Bow — Office: San Francisco, CA
Barry Jones — Office: No office assigned
Diane Murphy — Office: San Francisco, CA
Foon Yue Tseng — Office: NYC, NY
George Vanauf — Office: NYC, NY
Gerard Bondur — Office: No office assigned
Gerard Hernandez — Office: No office assigned
Jeff Firrelli — Office: San Francisco, CA
Julie Firrelli — Office: Boston, MA
Larry Bott — Office: No office assigned
Leslie Jennings — Office: San Francisco, CA
Leslie Thompson — Office: San Francisco, CA
Loui Bondur — Office: No office assigned
Mami Nishi — Office: Tokyo, Chiyoda-Ku
Martin Gerard — Office: No office assigned
Mary Patterson — Office: San Francisco, CA
Pamela Castillo — Office: No office assigned
Peter Marsh — Office: No office assigned
Steve Patterson — Office: Boston, MA
Tom King — Office: No office assigned
William Patterson — Office: No office assigned
Yoshimi Kato — Office: Tokyo, Chiyoda-Ku


## Customers and Their Orders (a One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' order numbers, order dates, and statuses.

In [3]:
# Your code here
cur.execute("""
    SELECT c.contactFirstName, c.contactLastName, o.orderNumber, o.orderDate, o.status
    FROM customers c
    JOIN orders o ON c.customerNumber = o.customerNumber
    ORDER BY c.contactLastName, c.contactFirstName, o.orderDate;
""")

results = cur.fetchall()
for first, last, order_num, order_date, status in results:
    print(f"{first} {last} — Order #{order_num}, Date: {order_date}, Status: {status}")


Paolo  Accorti — Order #10280, Date: 2004-08-17, Status: Shipped
Paolo  Accorti — Order #10293, Date: 2004-09-09, Status: Shipped
Rachel Ashworth — Order #10110, Date: 2003-03-18, Status: Shipped
Rachel Ashworth — Order #10306, Date: 2004-10-14, Status: Shipped
Rachel Ashworth — Order #10332, Date: 2004-11-17, Status: Shipped
Miguel Barajas — Order #10276, Date: 2004-08-02, Status: Shipped
Miguel Barajas — Order #10294, Date: 2004-09-10, Status: Shipped
Violeta Benitez — Order #10166, Date: 2003-10-21, Status: Shipped
Violeta Benitez — Order #10321, Date: 2004-11-04, Status: Shipped
Violeta Benitez — Order #10388, Date: 2005-03-03, Status: Shipped
Helen  Bennett — Order #10232, Date: 2004-03-20, Status: Shipped
Helen  Bennett — Order #10316, Date: 2004-11-01, Status: Shipped
Christina  Berglund — Order #10112, Date: 2003-03-24, Status: Shipped
Christina  Berglund — Order #10320, Date: 2004-11-03, Status: Shipped
Christina  Berglund — Order #10326, Date: 2004-11-09, Status: Shipped
Chri

## Customers and Their Payments (Another One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. Sort these results in descending order by the payment amount. 

In [6]:
# Your code here
cur.execute("""
    SELECT p.productName, p.productLine, SUM(od.quantityOrdered) AS total_quantity
    FROM products p
    JOIN orderdetails od ON p.productCode = od.productCode
    GROUP BY p.productCode
    ORDER BY total_quantity DESC;
""")
results = cur.fetchall()
for product_name, product_line, total_quantity in results:
    print(f"Product: {product_name} (Line: {product_line}) — Total Quantity Sold: {total_quantity}")


Product: 1992 Ferrari 360 Spider red (Line: Classic Cars) — Total Quantity Sold: 1808
Product: 1937 Lincoln Berline (Line: Vintage Cars) — Total Quantity Sold: 1111
Product: American Airlines: MD-11S (Line: Planes) — Total Quantity Sold: 1085
Product: 1941 Chevrolet Special Deluxe Cabriolet (Line: Vintage Cars) — Total Quantity Sold: 1076
Product: 1930 Buick Marquette Phaeton (Line: Vintage Cars) — Total Quantity Sold: 1074
Product: 1940s Ford truck (Line: Trucks and Buses) — Total Quantity Sold: 1061
Product: 1969 Harley Davidson Ultimate Chopper (Line: Motorcycles) — Total Quantity Sold: 1057
Product: 1957 Chevy Pickup (Line: Trucks and Buses) — Total Quantity Sold: 1056
Product: 1964 Mercedes Tour Bus (Line: Trucks and Buses) — Total Quantity Sold: 1053
Product: 1956 Porsche 356A Coupe (Line: Classic Cars) — Total Quantity Sold: 1052
Product: Corsair F4U ( Bird Cage) (Line: Planes) — Total Quantity Sold: 1051
Product: F/A 18 Hornet 1/72 (Line: Planes) — Total Quantity Sold: 1047
Pro

## Orders, Order Details, and Product Details (a Many-to-Many Join)

Select all of the customer contacts (first and last names) along with the product names, quantities, and date ordered for each of the customers and each of their orders. Sort these in descending order by the order date.

> Note: This will require joining 4 tables! This can be tricky! Give it a shot, and if you're still stuck, turn to the next section where you'll see how to write subqueries that can make complex queries such as this much simpler!

In [7]:
# Your code here
cur.execute("""
    SELECT 
        c.contactFirstName, 
        c.contactLastName, 
        p.productName, 
        od.quantityOrdered, 
        o.orderDate
    FROM 
        customers c
    JOIN 
        orders o ON c.customerNumber = o.customerNumber
    JOIN 
        orderdetails od ON o.orderNumber = od.orderNumber
    JOIN 
        products p ON od.productCode = p.productCode
    ORDER BY 
        o.orderDate DESC;
""")

results = cur.fetchall()
for first, last, product, quantity, order_date in results:
    print(f"{first} {last} ordered {quantity} of {product} on {order_date}")



Janine  Labrune ordered 38 of 1962 LanciaA Delta 16V on 2005-05-31
Janine  Labrune ordered 33 of 1957 Chevy Pickup on 2005-05-31
Janine  Labrune ordered 28 of 1998 Chrysler Plymouth Prowler on 2005-05-31
Janine  Labrune ordered 38 of 1964 Mercedes Tour Bus on 2005-05-31
Janine  Labrune ordered 19 of 1926 Ford Fire Engine on 2005-05-31
Janine  Labrune ordered 28 of 1992 Ferrari 360 Spider red on 2005-05-31
Janine  Labrune ordered 38 of 1940s Ford truck on 2005-05-31
Janine  Labrune ordered 55 of 1970 Dodge Coronet on 2005-05-31
Janine  Labrune ordered 49 of 1962 Volkswagen Microbus on 2005-05-31
Janine  Labrune ordered 31 of 1958 Chevy Corvette Limited Edition on 2005-05-31
Janine  Labrune ordered 41 of 1980’s GM Manhattan Express on 2005-05-31
Janine  Labrune ordered 11 of 1954 Greyhound Scenicruiser on 2005-05-31
Janine  Labrune ordered 18 of Diamond T620 Semi-Skirted Tanker on 2005-05-31
Diego  Freyre ordered 50 of 1952 Alpine Renault 1300 on 2005-05-31
Diego  Freyre ordered 49 of 19

## Summary

In this lab, you practiced your knowledge of one-to-many and many-to-many relationships!